In [ ]:
# извлечение имени героя и реплики из киносценария

import re
import pandas as pd
import os
from statistics import median
from tqdm import tqdm_notebook

def get_the_hero_and_lines(folder_input,folder_output):
    folder = os.listdir(folder_input)
    for filename in tqdm_notebook(folder):
        if 'txt' in filename:
            with open(folder_input+filename, encoding="utf8", errors='ignore') as f:
                text = f.read()
            lines = text.split('\n')

            replics = []
            small_let = re.compile('[a-z]')
            dig = re.compile('[\d]')
            stoplist = ["THE END.", "CONTINUED", "INT.", "EXT.", "FADE IN", "FADE OUT", "CUT TO:", 'DISSOLVE', "DRAFT", "MED", "CLOSE", "FULL", "REVISED"]
            
            
            replic = False

            for i, line in enumerate(lines):
                indent = (len(line) - len(line.lstrip()))

                # Если имя нашлось, мы в режиме "реплика"     
                if replic:
                    if not line.strip().startswith('(') and not line.strip().endswith(')') and len(line.strip()) > 0\
                        and not (re.search(dig, line) and len(line.strip())<3) and not stop_word:
                        if not first_phrase:
                            if indent == last_indent:
                                phrase += ' ' + line.strip().strip('*').strip()
                            else:
                                replic = False
                                phrase = phrase
                                replics.append([name, phrase])
                        else:
                            phrase = line.strip().strip('*').strip()
                        last_indent = indent
                        first_phrase = False

                # Условие "если содержит '(', то содержит и ')'    
                bad_line = False
                if ('(' in line and not ')' in line) or \
                    (')' in line and not '(' in line):
                    bad_line = True
                        
                #   Стоп-слова
                stop_word = False
                for word in stoplist:
                    if word in line:
                        stop_word = True
                        break

                # Поиск имени    
                if indent > 19 and indent < 40 and not bad_line and not line.strip().startswith('(') \
                and not re.search(small_let, line) and not stop_word \
                and not re.search(dig, line):
                    testname = line.strip().strip('*').strip()
                    
                    if len(testname) > 0:
                        name = testname
                        replic = True
                        first_phrase = True
                    

            df = pd.DataFrame(replics, columns = ['name', 'phrase'])
            df.dropna().to_csv(folder_output+filename.replace('.txt','')+".csv", index=False)